In [2]:
import pandas as pd
import numpy as np
import random
import string

## Section-1: Dataset Manipulation
---
The following main goals are achieved in this section:
  - Dataset is loaded
  - Uninteresting lines are deleted from the dataset
  - Lines are converted into a list of string for further processing

In [6]:
from datasets import load_dataset

dataset = load_dataset("phamson02/vietnamese-poetry-corpus")

Found cached dataset csv (/home/pham-son/.cache/huggingface/datasets/phamson02___csv/phamson02--vietnamese-poetry-corpus-8d629a9d62b1c63b/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 1/1 [00:00<00:00, 295.02it/s]


In [8]:
df = pd.DataFrame(dataset["train"])
df.head()

,content,title,url,genre,period,specific_genre,author
0,"đẩy hoa dun lá khỏi tay trời , <\n> nghĩ lại t...",Gửi cô Trương Quỳnh Như bài 2,https://www.thivien.net/,bảy chữ,Tây Sơn,thất ngôn bát cú,Phạm Thái
1,"từ chốn thềm cung trộm dấu hương , <\n> dễ xui...",Gửi cô Trương Quỳnh Như bài 1,https://www.thivien.net/,bảy chữ,Tây Sơn,thất ngôn bát cú,Phạm Thái
2,"hợi sang xanh biếc ngọn đèn tàn , <\n> gượng đ...",Giờ hợi,https://www.thivien.net/,bảy chữ,Tây Sơn,thất ngôn bát cú,Phạm Thái
3,"giờ tuất canh sang được mấy phần , <\n> càng t...",Giờ tuất,https://www.thivien.net/,bảy chữ,Tây Sơn,thất ngôn bát cú,Phạm Thái
4,"trời xanh cao thẳm mấy tầng khơi , <\n> nỡ để ...",Khóc cô Trương Quỳnh Như bài 1,https://www.thivien.net/,bảy chữ,Tây Sơn,thất ngôn bát cú,Phạm Thái


In [9]:
genre_list = df['genre'].tolist()
genre_list = list( dict.fromkeys(genre_list) )

print(genre_list)

['bảy chữ', 'thơ tự do', 'lục bát', 'năm chữ', 'bốn chữ', 'tám chữ', 'sáu chữ']


In [10]:
df = df.loc[df['genre'] == 'lục bát']
print(len(df))

89943


In [11]:
df = df.sample(frac=0.12)
print(len(df))

10793


In [12]:
lines = df['content'].tolist ()
print(lines [:5])

['tiếng ve rền dĩ hè về <\n> mùa hoa phượng đỏ gọi hè hai mươi <\n> mênh mang thân thiện nụ cười <\n> về vui nghỉ mát tình người quê em <\n> <\n> biết bao nét đẹp mà xem <\n> tấm hình kỷ niệm ai đem hữu tình <\n> nhớ về quê mẹ thân thương <\n> gợi thương gợi nhớ quê hương bến làng', 'em là con gái nhà quê <\n> quanh năm cày cấy sớm khuya ruộng vườn <\n> tay bùn trân nấm sớm hôm <\n> chẳng biết thành thị phồn vinh thế nào', 'ta thương phận đá mồ côi <\n> cô đơn xanh giữa cõi đời mênh mông <\n> <\n> đời lăn lóc đá buồn không <\n> hoang vu gió quất cho lòng đá đau <\n> <\n> thề nguyền tạc đến ngàn sau <\n> trăm năm lỗi hẹn lạc nhau có về <\n> <\n> đá mòn mỏi đá tái tê <\n> ta ru đá đá ru về thiên thu', 'trâu vàng gõ cửa từng nhà <\n> mùa màng tươi tốt thế là ấm no <\n> của cải làm được đầy kho <\n> tiền vào đầy két trời cho kiếm tiền <\n> <\n> đầu năm chồng vợ cưới liền <\n> bảo nhau tích cưc kiếm tiền thật chăm <\n> hêt dịch xây lầu cuối năm <\n> nội thất xắm đủ rồi chăm thằng dần <\n> <

In [13]:
# The length of lines generated based on HMM is restricted to this value.
# It is a safeguard to protect the code from a potential infinite loop
MAX_LINE_LENGTH = 8

# This is used to check the equality of two floating point values. If
# their difference is less than this value, they are considered equal
FLOAT_DELTA = 0.0001

# This special string is used to mark the end of a sentence
END_TOKEN = 'endl'

### Section-2.2: Helper Functions for Text-Parsing

In [14]:
# Remove white-spaces and punctuations from a line and convert it
# into a list of tokens
def tokenize (line):
    baseline = line.strip ().lower ()
    tokens = ''.join ([x for x in baseline if x not in string.punctuation]).split ()
    return tokens

# Add a pairing to dictionary
def insert_link (dictionary, key, value, debug = False):
    if key not in dictionary:
        dictionary [key] = []
    if debug: print(key, dictionary [key])
    dictionary [key].append (value)

# Convert list to probability values
def to_probability (chain):
    frequencies = {}
    probabilities = {}
    num_of_words = len (chain)

    for word in chain:
        frequencies [word] = frequencies.get (word, 0) + 1

    for word, frequency in frequencies.items ():
        probabilities [word] = round (float (frequency) / num_of_words, 3)

    return probabilities

### Section-2.3: Main Function for Building the Markov Model

In [15]:
def build_markov_model (corpus, first_order_markov_chain, second_order_markov_chain):
    # This is a dictionary of words which have been used to
    # start a line in Shakespeare's plays
    words = []

    for line in corpus:
        tokens = tokenize (line)
        num_of_tokens = len (tokens)

        for idx in range (num_of_tokens):
            token = tokens [idx]

            if idx == 0:
                words.append (token)

                # We are not interested in the first word of a
                # line since nothing precedes it
                continue

            # Populate first-order markov chain
            last_token = tokens [idx - 1]
            insert_link (first_order_markov_chain, last_token, token)

            # The second word in a line can only have a first-level
            # markov chain since there is only a single word before it
            if idx == 1:
                continue

            # The last pair of word of a line is special. We want
            # to chain it with 'END'; to help in finishing a line
            # during predicitions
            if idx == num_of_tokens - 1:
                insert_link (second_order_markov_chain, (last_token, token), END_TOKEN)

            # Populate second-order markov chain
            second_last_token = tokens [idx - 2]
            insert_link (second_order_markov_chain, (second_last_token, last_token), token)

    # Convert first-order markov chain to probability values
    for word, chain in first_order_markov_chain.items ():
        first_order_markov_chain [word] = to_probability (chain)

    # Convert first-order markov chain to probability values
    for pair, chain in second_order_markov_chain.items ():
        second_order_markov_chain [pair] = to_probability (chain)

    print ('[STATUS] Successfully built Markov Model on Corpus!\n')
    return list (set (words))

### Section-2.4: Helpers Functions for using the Markov Model for Text-Generation from the Corpus

In [16]:
# Pick next word from the second-order markov chain. It should be the
# highest probability one. If multiple such words exist, randomly pick one
def predict_next_word (key, dictionary, debug = False):
    max_probability = 0.0
    most_probable_words = []

    for next_word, probability in dictionary.items ():
        if probability > max_probability:
            max_probability = probability
            most_probable_words = [next_word]
        elif max_probability - probability < FLOAT_DELTA:
            most_probable_words.append (next_word)

    if debug: print (key, most_probable_words)
    return random.choice (most_probable_words)

# Randomly pick a word that can follow; from the first-order markov chain
def pick_next_word (key, dictionary, debug = False):
    if debug: print(dictionary)
    return random.choice (list(dictionary.keys()))

# Generate text based on corpus
def write_line (start_word, markov_chain_one, markov_chain_two):
    line = []
    word = start_word.lower ()

    if word not in markov_chain_one.keys ():
        return 0

    line.append (word)
    next_word = pick_next_word (start_word, markov_chain_one [start_word])
    line.append (next_word)

    n = 0
    while n < MAX_LINE_LENGTH:
        next_next_word = predict_next_word ((word, next_word), markov_chain_two [(word, next_word)])

        if next_next_word == END_TOKEN:
            return ' '.join (line)

        word = next_word
        next_word = next_next_word
        line.append (next_next_word)
        n += 1

# Write a Shakespeare play of given length
def write_play (hints, mc1, mc2):
    string_generated = ""
    for word in hints:
        line = write_line (word, mc1, mc2)
        if (line):
          line += "\n"
          string_generated += line
    return string_generated

### Section-2.5: User API for Text Prediction Given a Sequence of Words

In [17]:
def predict_next (sequence, mc1, mc2):
    # Sanity checks
    sequence = sequence.strip ()
    if (sequence == ""):
        raise ValueError('Sequence must not be an empty string. Please retry!')

    tokens = tokenize (sequence)
    line = ''
    for token in reversed (tokens):
        line = write_line (token, mc1, mc2)

        if line:
            break

    return line + "\n"

In [18]:
%%time

# This is the first order markov chain. It chains a word
# with the word(s) that can come after it
mc_odr1 = {}

# This is the second order markov chain. It chains a pair
# of words with word(s) that can follow it
mc_odr2 = {}

words = build_markov_model(lines, mc_odr1, mc_odr2)

[STATUS] Successfully built Markov Model on Corpus!

CPU times: user 2.84 s, sys: 25.1 ms, total: 2.87 s
Wall time: 2.86 s


In [19]:
play_length = 150
hints = [random.choice (words) for x in range (play_length)]
poem = write_play (hints, mc_odr1, mc_odr2)
print(poem)

chưa ánh mắt nụ cười của em
vườn nghĩ ra phận mình
vụng nhớ thầm nhớ trộm ai hay
nún môi gọt bút phiêu rằm thơ tình
quà ngày này năm xưa
buông mát mẻ đêm trường nhớ ai
ngõ cây đào trước gió đung đưa
chúc huynh tỷ muội tặng nàng
nẻo xưa nhà bên nhau
vọng đẩy lui nỗi buồn
mỉm cười em đến bên tôi
tre xạc xào lá rụng rơi giọt buồn
cây bút viết nhiều lời thơ yêu thương
biên giữ cho em
cưới tươi hồng ban mai
buồn ước mơ
ngôi vào giữa lòng đất mẹ quê hương
trên áo cồn cào nhớ nhung
thảng vừa nhớ nhung
xõa lòng anh vẫn nhớ vẫn chờ đợi ai
bát rộn ràng tiếng trống xa cắc tòm
nông cụ lao động sớm hôm chăm làm vui
nợ trăng gió ngàn đêm cho vơi nỗi niềm
đoàn cháu con
nông hợp lực ta cùng chung một nhà
chè cũng đàn bà ” cho đời thêm vui
ngẩn cam sành lột luôn chiều lên cao
thoắt đường dài em đi để lại cho người
suối rong chơi bên mẹ hiền
bằng trợ giáo rạng danh nước nhà
ngân vẳng câu hát ngọt ngào
quay về
chẳng thành duyên tơ hồng se duyên tình
búp bê thương đau yêu nhau
vao trong mơ em đi để lại ch

In [20]:
import re
print(len(re.findall(r'\w+', poem)))

348


In [22]:
import pickle
import spacy
import numpy as np
import torch
from evaluate import load
from collections import defaultdict
from nltk.util import ngrams


def compute_perplexity(all_texts_list):
    torch.cuda.empty_cache()
    perplexity = load("perplexity", module_type="metric")
    # max sequence length and batch size are limited to 256 and 2, respectively, to avoid OOM bug
    resized_all_texts_list = [text[:256] for text in all_texts_list]
    results = perplexity.compute(predictions=resized_all_texts_list, model_id="vinai/bartpho-word", device='cuda', batch_size=2)
    return results['mean_perplexity']

def compute_wordcount(all_texts_list):
    wordcount = load("word_count")
    wordcount = wordcount.compute(data=all_texts_list)
    return wordcount['unique_words']


def compute_diversity(all_texts_list):
    ngram_range = [2, 3, 4]

    tokenizer = spacy.load("en_core_web_sm").tokenizer
    token_list = []
    for sentence in all_texts_list:
        token_list.append([str(token) for token in tokenizer(sentence)])
    ngram_sets = {}
    ngram_counts = defaultdict(int)

    metrics = {}
    for n in ngram_range:
        ngram_sets[n] = set()
        for tokens in token_list:
            ngram_sets[n].update(ngrams(tokens, n))
            ngram_counts[n] += len(list(ngrams(tokens, n)))
        metrics[f'{n}gram_repitition'] = (1 - len(ngram_sets[n]) / ngram_counts[n])
    diversity = 1
    for val in metrics.values():
        diversity *= (1 - val)
    metrics['diversity'] = diversity
    return metrics

list_of_string = []
# generate words
n = 256
for i in range(150):
    play_length = 150
    hints = [random.choice (words) for x in range (play_length)]
    poem = write_play (hints, mc_odr1, mc_odr2)
    list_of_string.append(poem)
list_of_string = [i.replace("\n", ".") for i in list_of_string]
print(list_of_string[0])
list_of_string = list(filter(lambda a: a != " ", list_of_string))

perplexity = compute_perplexity(list_of_string)
wordcount = compute_wordcount(list_of_string)
diversity = compute_diversity(list_of_string)
print("Perplexity:", perplexity)
print("Wordcount:", wordcount)
print("Diversity:", diversity)


đống tim hồng của em.cấm mất rồi.thức chưa quên một người.đồ môn nghệ thuật chuyên chuân vẹn tròn.lãng chứa chan tình người.với ngổn ngang trăm mối tơ vò với ai.cửu trên đời chẳng quên.hè bốn mươi năm lẻ bóng cô đơn.cắm xuống lại ngày xưa.quanh rối rắm tơ vò với ai.phan thi nữ ngọt ngào.sầu hay là em đã có chồng hay chưa.sắp biệt nhớ nhung.nào sen quê hương.làng ngày xưa.đỏ năm xưa.rau giỗ người.rã nhau.buổi đoạn trường ai hay.nửa khuyết tái tê cõi lòng.tha kiếp này.thím đương thì.lối đê trót vì thơ.hết em về với mẹ cha.tới mình tuổi thơ.níu bồi hồi nhớ thương.đầm để dành cho nhau.chèo xưa hát xứ đông cho ta nhớ người.san tả tơi thân xác héo hon.ngót nghét tám mươi năm lẻ bóng cô liêu.vu khi thân nát hồn đau con tim.tủi chẳng ai có nhớ không tên thuở nào.kiếp loài ta nghĩa tình.phím chừ nằm nỗi đau.củ mì từ đây.dáng những hình hài của con.cành nguyên trinh cho đời thêm vui.nghe lấy những lời yêu thương.rung tâm hồn.nhánh thương đau yêu nhau.cuộn tràn tình thương dạt dào tình thơ.hồng c

Some weights of the model checkpoint at vinai/bartpho-word were not used when initializing MBartForCausalLM: ['encoder.layers.9.final_layer_norm.bias', 'encoder.layers.6.final_layer_norm.bias', 'encoder.layer_norm.weight', 'encoder.layers.0.self_attn.k_proj.weight', 'encoder.layers.6.fc1.weight', 'encoder.embed_positions.weight', 'encoder.layers.11.self_attn.v_proj.bias', 'encoder.layers.7.self_attn_layer_norm.bias', 'encoder.layers.8.self_attn.out_proj.weight', 'encoder.layers.4.self_attn.v_proj.bias', 'encoder.layers.11.fc1.weight', 'encoder.layers.6.self_attn_layer_norm.bias', 'encoder.layers.10.self_attn_layer_norm.weight', 'encoder.layers.6.self_attn_layer_norm.weight', 'encoder.layers.2.fc2.weight', 'encoder.layers.0.self_attn_layer_norm.weight', 'encoder.layers.7.self_attn.out_proj.weight', 'encoder.layers.6.final_layer_norm.weight', 'encoder.layers.8.final_layer_norm.bias', 'encoder.layers.6.fc1.bias', 'encoder.layers.10.final_layer_norm.weight', 'encoder.layers.11.self_attn_la

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

Following code demonstrates **text prediction given a sequence**: